# Regressão Linear - Modelo Inicial

## Configurações Iniciais

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import requests as req
from sklearn.linear_model import LinearRegression
import geopandas as gpd
import statsmodels.api as sm

In [ ]:
import os

# Ajusta o caminho relativo para o arquivo de dados
data_path = os.path.join("..", "data", "merged_df.pkl")
merged_df = pd.read_pickle(data_path)

merged_df.head()

## Análise com 1 variável - Bolsa Família

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Prepare data: drop NaNs for the relevant columns
df_lr = merged_df[['perc_bolsa_familia', 'voto_lula']].dropna()

X = df_lr[['perc_bolsa_familia']].values
y = df_lr['voto_lula'].values

# Split into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=None
)

# Train the model on training data
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on both sets
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Calculate metrics for both training and test sets
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

print("=== MODELO COM TRAIN/TEST SPLIT ===\n")
print(f"Intercept: {model.intercept_:.4f}")
print(f"Coefficient: {model.coef_[0]:.4f}")

print(f"\n=== PERFORMANCE NO CONJUNTO DE TREINO ===")
print(f"Tamanho do treino: {len(X_train)} observações")
print(f"R² (treino): {train_r2:.4f}")
print(f"MAE (treino): {train_mae:.4f}")

print(f"\n=== PERFORMANCE NO CONJUNTO DE TESTE ===")
print(f"Tamanho do teste: {len(X_test)} observações")
print(f"R² (teste): {test_r2:.4f}")
print(f"MAE (teste): {test_mae:.4f}")

print(f"\n=== AVALIAÇÃO DE OVERFITTING ===")
r2_diff = train_r2 - test_r2
mae_diff = test_mae - train_mae

print(f"Diferença R² (treino - teste): {r2_diff:.4f}")
print(f"Diferença MAE (teste - treino): {mae_diff:.4f}")

if r2_diff > 0.05:
    print("⚠️ ALERTA: Possível overfitting (grande diferença no R²)")
elif r2_diff > 0.02:
    print("⚠️ ATENÇÃO: Leve overfitting detectado")
else:
    print("✅ OK: Sem sinais de overfitting")

### Interpretação dos Parâmetros da Regressão Linear

### **Equação do Modelo**
```
% Voto Lula = Intercept + (Coefficient × % Bolsa Família)
% Voto Lula = 29.14 + (1.85 × % Bolsa Família)
```

### **Intercept (Intercepto) = 29.14**
- **O que significa**: O valor previsto de % Voto Lula quando % Bolsa Família = 0%
- **Interpretação prática**: Em um município hipotético com 0% de famílias no Bolsa Família, o modelo prevê que Lula teria 29.14% dos votos
- **Contexto**: Representa a "base" de votos em Lula independente do Bolsa Família

### **Coefficient (Coeficiente) = 1.85**
- **O que significa**: Para cada 1 ponto percentual de aumento na % de famílias no Bolsa Família, o % de voto em Lula aumenta 1.85 pontos percentuais
- **Interpretação**: O Bolsa Família tem um efeito amplificador - cada 1% de cobertura adicional resulta em quase 2% a mais de votos para Lula
- **Significado político**: Mostra uma relação muito forte entre o programa social e o voto

In [ ]:
# Exemplos práticos usando os parâmetros da regressão

print("=== EXEMPLOS PRÁTICOS DO MODELO ===\n")
print("Equação: % Voto Lula = 29.14 + (1.85 × % Bolsa Família)\n")

# Exemplos de predição
scenarios = [
    ("Município com baixa cobertura", 5),
    ("Município com cobertura média", 15), 
    ("Município com alta cobertura", 30),
    ("Município com cobertura muito alta", 50)
]

for desc, perc_bf in scenarios:
    voto_previsto = model.intercept_ + model.coef_[0] * perc_bf
    print(f"{desc} ({perc_bf}% Bolsa Família):")
    print(f"  → Voto Lula previsto: {voto_previsto:.1f}%")
    
# Interpretação do coeficiente
print(f"\n=== INTERPRETAÇÃO DO COEFICIENTE ===")
print(f"Coeficiente = {model.coef_[0]:.2f}")
print(f"→ A cada 1% adicional de famílias no Bolsa Família:")
print(f"  • O voto em Lula aumenta {model.coef_[0]:.2f} pontos percentuais")
print(f"→ A cada 10% adicionais de famílias no Bolsa Família:")
print(f"  • O voto em Lula aumenta {model.coef_[0] * 10:.1f} pontos percentuais")

# Comparação com dados reais
print(f"\n=== COMPARAÇÃO COM EXTREMOS DOS DADOS ===")
min_bf = merged_df['perc_bolsa_familia'].min()
max_bf = merged_df['perc_bolsa_familia'].max()

pred_min = model.intercept_ + model.coef_[0] * min_bf
pred_max = model.intercept_ + model.coef_[0] * max_bf

print(f"Menor % Bolsa Família observada: {min_bf:.1f}% → Lula previsto: {pred_min:.1f}%")
print(f"Maior % Bolsa Família observada: {max_bf:.1f}% → Lula previsto: {pred_max:.1f}%")
print(f"Diferença total prevista: {pred_max - pred_min:.1f} pontos percentuais")

In [ ]:
# Recreate and save the figure with a simplified approach
plt.figure(figsize=(12, 6))

# Plot training data
plt.scatter(X_train[:, 0], y_train, alpha=0.4, label='Dados de Treino', color='lightblue', s=20)

# Plot test data
plt.scatter(X_test[:, 0], y_test, alpha=0.6, label='Dados de Teste', color='orange', s=15, marker='^')

# Create regression line
x_range = np.linspace(X_train[:, 0].min(), X_train[:, 0].max(), 100)
y_line = model.predict(x_range.reshape(-1, 1))
plt.plot(x_range, y_line, color='red', linewidth=2, label='Regressão Linear (Treino)')

plt.xlabel('% de Famílias Beneficiárias do Bolsa Família')
plt.ylabel('% Voto Lula')
plt.title('Regressão Linear: Dados de Treino vs Teste')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()

# # Save immediately
# # Salva a figura como PNG com alta resolução
# filename = "regressao_linear_train_test.png"
# plt.savefig(filename, dpi=300, bbox_inches='tight')
#
# # Verifica se o arquivo foi criado com sucesso
# import os
# if os.path.exists(filename):
#     print(f"✅ SUCCESS: Figure saved as {filename}")
#     print(f"File size: {os.path.getsize(filename)} bytes")
# else:
#     print(f"❌ ERROR: Failed to save {filename}")

plt.show()

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

y_pred = model.predict(X)
r2 = r2_score(y, y_pred)
mse = mean_squared_error(y, y_pred)
mae = mean_absolute_error(y, y_pred)

print(f"R² score: {r2:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")

### Interpretação das Métricas de Regressão

### **R² Score (Coeficiente de Determinação)**
- **Significado**: Proporção da variância explicada pelo modelo
- **Range**: 0 a 1 (valores próximos a 1 são melhores)
- **Interpretação**: 65.5% da variação no voto em Lula é explicada pela variação no Bolsa Família

### **Mean Squared Error (MSE) - Erro Quadrático Médio**
- **Significado**: Média dos quadrados dos erros de predição
- **Unidade**: (% pontos)² 
- **Interpretação**: Penaliza mais os erros grandes
- **Melhor**: Valores menores são melhores

### **Mean Absolute Error (MAE) - Erro Absoluto Médio**
- **Significado**: Média dos valores absolutos dos erros
- **Unidade**: Pontos percentuais
- **Interpretação**: Em média, o modelo erra por X pontos percentuais
- **Melhor**: Valores menores são melhores

In [ ]:
# Vamos contextualizar as métricas com os dados reais
print("=== CONTEXTUALIZANDO AS MÉTRICAS ===\n")

print(f"R² Score: {r2:.4f}")
print(f"→ O modelo explica {r2*100:.1f}% da variação no voto em Lula")
print(f"→ {(1-r2)*100:.1f}% da variação é devido a outros fatores\n")

print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"→ Raiz do MSE (RMSE): {np.sqrt(mse):.2f} pontos percentuais")
print(f"→ Erro 'típico' do modelo: ±{np.sqrt(mse):.1f}pp\n")

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"→ Em média, o modelo erra {mae:.1f} pontos percentuais")
print(f"→ Se um município tem 50% de voto Lula, o modelo pode prever entre {50-mae:.1f}% e {50+mae:.1f}%\n")

# Contexto dos dados
voto_lula_range = merged_df['voto_lula'].max() - merged_df['voto_lula'].min()
print(f"=== CONTEXTO DOS DADOS ===")
print(f"Range do % Voto Lula: {merged_df['voto_lula'].min():.1f}% a {merged_df['voto_lula'].max():.1f}%")
print(f"Amplitude total: {voto_lula_range:.1f} pontos percentuais")
print(f"MAE como % da amplitude: {(mae/voto_lula_range)*100:.1f}%")
print(f"→ O erro médio representa {(mae/voto_lula_range)*100:.1f}% da variação total dos dados")

### O que significa "r" nas análises estatísticas?

### **"r" = Coeficiente de Correlação de Pearson**

**Definição**: O "r" representa a força e direção da relação linear entre duas variáveis.

**Características do "r"**:
- **Range**: -1 ≤ r ≤ +1
- **Sinal**: 
  - r > 0: correlação positiva (quando uma variável aumenta, a outra também aumenta)
  - r < 0: correlação negativa (quando uma variável aumenta, a outra diminui)
  - r = 0: sem correlação linear

**Interpretação da força da correlação**:
- |r| = 0.00 a 0.30: correlação fraca
- |r| = 0.30 a 0.70: correlação moderada  
- |r| = 0.70 a 1.00: correlação forte

### **Exemplos do nosso estudo**:
- r = 0.8095 entre % Bolsa Família e % Voto Lula
  - **Interpretação**: Correlação positiva muito forte
  - **Significado**: Municípios com maior % de famílias no Bolsa Família tendem a ter maior % de votos em Lula

### **Diferença entre "r" e "R²"**:
- **r**: Mede a correlação (-1 a +1)
- **R²**: Mede a proporção da variância explicada (0 a 1)
- **Relação**: R² = r² (R² é o quadrado do r)

**No nosso modelo**:
- r ≈ √0.6553 ≈ 0.81 (correlação)
- R² = 0.6553 (65.5% da variância explicada)

In [ ]:
# Exemplos práticos do "r" no nosso estudo

print("=== EXEMPLOS DE 'r' (CORRELAÇÃO DE PEARSON) NO NOSSO ESTUDO ===\n")

# Recalcular algumas correlações para demonstrar
if 'perc_bolsa_familia' in merged_df.columns and 'voto_lula' in merged_df.columns:
    r_bolsa_lula = merged_df[['perc_bolsa_familia', 'voto_lula']].corr().iloc[0,1]
    print(f"1. % Bolsa Família vs % Voto Lula:")
    print(f"   r = {r_bolsa_lula:.4f}")
    print(f"   Interpretação: Correlação positiva MUITO FORTE")
    print(f"   Significado: Quanto maior o Bolsa Família, maior o voto em Lula\n")

if 'pib per capita' in merged_df.columns:
    r_pib_lula = merged_df[['pib per capita', 'voto_lula']].corr().iloc[0,1]
    print(f"2. PIB per capita vs % Voto Lula:")
    print(f"   r = {r_pib_lula:.4f}")
    if r_pib_lula < 0:
        print(f"   Interpretação: Correlação negativa")
        print(f"   Significado: Quanto maior a renda, menor o voto em Lula")
    else:
        print(f"   Interpretação: Correlação positiva")
        print(f"   Significado: Quanto maior a renda, maior o voto em Lula")
    print()

r_pop_lula = merged_df[['POPULAÇÃO ESTIMADA', 'voto_lula']].corr().iloc[0,1]
print(f"3. População vs % Voto Lula:")
print(f"   r = {r_pop_lula:.4f}")
if abs(r_pop_lula) < 0.3:
    print(f"   Interpretação: Correlação FRACA")
elif abs(r_pop_lula) < 0.7:
    print(f"   Interpretação: Correlação MODERADA")
else:
    print(f"   Interpretação: Correlação FORTE")

print(f"\n=== RELAÇÃO ENTRE r E R² ===")
print(f"Se r = {r_bolsa_lula:.4f}, então:")
print(f"R² = r² = ({r_bolsa_lula:.4f})² = {r_bolsa_lula**2:.4f}")
print(f"Isso significa que {r_bolsa_lula**2*100:.1f}% da variação é explicada pelo modelo")

##  Testando Pib per Capita

In [ ]:
# Multiple Linear Regression with Train/Test Split: Bolsa Família + PIB per capita
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

# Prepare data: drop NaNs for all relevant columns
df_multiple = merged_df[['perc_bolsa_familia', 'pib per capita', 'voto_lula']].dropna()

# Features (X) and target (y)
X_multiple = df_multiple[['perc_bolsa_familia', 'pib per capita']].values
y_multiple = df_multiple['voto_lula'].values

# Split into train and test sets (80% train, 20% test)
X_train_mult, X_test_mult, y_train_mult, y_test_mult = train_test_split(
    X_multiple, y_multiple, test_size=0.2, random_state=42
)

# Train the multiple regression model on training data
model_multiple = LinearRegression()
model_multiple.fit(X_train_mult, y_train_mult)

# Make predictions on both sets
y_train_pred_mult = model_multiple.predict(X_train_mult)
y_test_pred_mult = model_multiple.predict(X_test_mult)

# Calculate metrics for both training and test sets
train_r2_mult = r2_score(y_train_mult, y_train_pred_mult)
test_r2_mult = r2_score(y_test_mult, y_test_pred_mult)
train_mae_mult = mean_absolute_error(y_train_mult, y_train_pred_mult)
test_mae_mult = mean_absolute_error(y_test_mult, y_test_pred_mult)

print("=== MODELO MÚLTIPLO COM TRAIN/TEST SPLIT ===\n")
print(f"Intercept: {model_multiple.intercept_:.4f}")
print(f"Coeficiente Bolsa Família: {model_multiple.coef_[0]:.4f}")
print(f"Coeficiente PIB per capita: {model_multiple.coef_[1]:.6f}")
print(f"\nEquação do modelo:")
print(f"% Voto Lula = {model_multiple.intercept_:.2f} + ({model_multiple.coef_[0]:.2f} × % Bolsa Família) + ({model_multiple.coef_[1]:.6f} × PIB per capita)")

print(f"\n=== PERFORMANCE NO CONJUNTO DE TREINO ===")
print(f"Tamanho do treino: {len(X_train_mult)} observações")
print(f"R² (treino): {train_r2_mult:.4f}")
print(f"MAE (treino): {train_mae_mult:.4f}")

print(f"\n=== PERFORMANCE NO CONJUNTO DE TESTE ===")
print(f"Tamanho do teste: {len(X_test_mult)} observações")
print(f"R² (teste): {test_r2_mult:.4f}")
print(f"MAE (teste): {test_mae_mult:.4f}")

print(f"\n=== AVALIAÇÃO DE OVERFITTING ===")
r2_diff_mult = train_r2_mult - test_r2_mult
mae_diff_mult = test_mae_mult - train_mae_mult

print(f"Diferença R² (treino - teste): {r2_diff_mult:.4f}")
print(f"Diferença MAE (teste - treino): {mae_diff_mult:.4f}")

if r2_diff_mult > 0.05:
    print("⚠️ ALERTA: Possível overfitting (grande diferença no R²)")
elif r2_diff_mult > 0.02:
    print("⚠️ ATENÇÃO: Leve overfitting detectado")
else:
    print("✅ OK: Sem sinais de overfitting")

print(f"\n=== COMPARAÇÃO COM MODELO SIMPLES ===")
print(f"Modelo simples (apenas Bolsa Família) - R² teste: {test_r2:.4f}")
print(f"Modelo múltiplo (+ PIB per capita) - R² teste: {test_r2_mult:.4f}")
print(f"Melhoria no R² teste: {test_r2_mult - test_r2:.4f} ({((test_r2_mult - test_r2)/test_r2)*100:.1f}% de aumento)")

### 📊 Análise: Por que PIB per capita adicionou pouco valor?

### **Resultado:**
- **Melhoria**: Apenas 0.7% no R² (de 66.30% para 67.00%)
- **Conclusão**: PIB per capita não é uma variável muito útil neste modelo

### **Possíveis explicações:**

1. **Redundância com Bolsa Família**
   - Bolsa Família já captura informação socioeconômica
   - PIB per capita pode estar medindo algo similar

2. **Correlação moderada**
   - r = -0.38 entre Bolsa Família e PIB per capita
   - Parte da informação já está "incluída" no Bolsa Família

3. **Efeito não-linear**
   - Relação entre renda e voto pode não ser linear
   - Modelo linear simples não captura complexidade

4. **Variável menos importante**
   - Programas sociais podem ter impacto direto maior
   - Renda pode ser menos relevante que benefícios diretos

### **Lição aprendida:**
- Nem sempre "mais variáveis = modelo melhor"
- Melhoria de <2% no R² geralmente não justifica complexidade adicional
- Bolsa Família sozinho já é um preditor muito forte (R² = 66.3%)

In [ ]:
# Quantificando por que PIB per capita adiciona pouco valor

print("=== ANÁLISE DETALHADA: PIB PER CAPITA ===\n")

# 1. Correlação entre as variáveis
corr_bf_pib = merged_df[['perc_bolsa_familia', 'pib per capita']].corr().iloc[0,1]
corr_pib_lula = merged_df[['pib per capita', 'voto_lula']].corr().iloc[0,1]
corr_bf_lula = merged_df[['perc_bolsa_familia', 'voto_lula']].corr().iloc[0,1]

print("1. CORRELAÇÕES:")
print(f"Bolsa Família vs PIB per capita: {corr_bf_pib:.4f}")
print(f"PIB per capita vs Voto Lula: {corr_pib_lula:.4f}")
print(f"Bolsa Família vs Voto Lula: {corr_bf_lula:.4f}")

# 2. Comparação de força das correlações
print(f"\n2. FORÇA DAS VARIÁVEIS:")
print(f"Bolsa Família: MUITO FORTE (r = {abs(corr_bf_lula):.3f})")
print(f"PIB per capita: MODERADA (r = {abs(corr_pib_lula):.3f})")
print(f"→ Bolsa Família é {abs(corr_bf_lula)/abs(corr_pib_lula):.1f}x mais correlacionado")

# 3. Análise da melhoria
improvement = test_r2_mult - test_r2
print(f"\n3. MELHORIA NO MODELO:")
print(f"Melhoria absoluta: {improvement:.4f} ({improvement*100:.1f} pontos percentuais)")
print(f"Melhoria relativa: {(improvement/test_r2)*100:.1f}%")

if improvement < 0.01:
    print("❌ CONCLUSÃO: Melhoria INSIGNIFICANTE")
elif improvement < 0.02:
    print("⚠️ CONCLUSÃO: Melhoria MUITO PEQUENA")
elif improvement < 0.05:
    print("✅ CONCLUSÃO: Melhoria MODESTA")
else:
    print("🎯 CONCLUSÃO: Melhoria SIGNIFICATIVA")

# 4. Recomendação
print(f"\n4. RECOMENDAÇÃO:")
print(f"PIB per capita adiciona apenas {improvement*100:.1f}pp de explicação.")
print(f"Para análise política, modelo simples (só Bolsa Família) pode ser preferível:")
print(f"• Mais interpretável")
print(f"• Quase a mesma performance (R² = {test_r2:.3f})")
print(f"• Evita redundância")

# 5. Próximas variáveis sugeridas
print(f"\n5. PRÓXIMAS VARIÁVEIS A TESTAR:")
print(f"• Região (dummy variables) - pode ter impacto maior")
print(f"• População/densidade - diferentes dinâmicas urbano/rural")
print(f"• Interação Bolsa Família × Região - efeitos regionais")

### ⚠️ Cuidados ao Adicionar Mais Variáveis ao Modelo

### **1. Multicolinearidade**
- **Problema**: Variáveis independentes muito correlacionadas entre si
- **Exemplo**: PIB per capita e escolaridade geralmente são altamente correlacionados
- **Consequências**: Coeficientes instáveis, difíceis de interpretar
- **Como detectar**: Matriz de correlação, VIF (Variance Inflation Factor)

### **2. Overfitting (Sobreajuste)**
- **Problema**: Modelo muito complexo que "decora" os dados ao invés de aprender padrões
- **Sintomas**: R² muito alto, mas performance ruim em dados novos
- **Solução**: Cross-validation, dividir dados em treino/teste

### **3. Maldição da Dimensionalidade**
- **Problema**: Muitas variáveis em relação ao número de observações
- **Regra prática**: Pelo menos 10-15 observações por variável
- **Nosso caso**: 5.570 observações → máximo ~350-550 variáveis

### **4. Causalidade vs Correlação**
- **Problema**: Correlação não implica causalidade
- **Exemplo**: Bolsa Família pode estar correlacionado com pobreza, que é a verdadeira causa
- **Solução**: Pensar na teoria por trás das relações

### **5. Variáveis Confundidoras**
- **Problema**: Variável omitida que afeta tanto X quanto Y
- **Exemplo**: Região pode afetar tanto Bolsa Família quanto voto
- **Solução**: Incluir variáveis de controle importantes

### **6. Interpretabilidade**
- **Trade-off**: Mais variáveis = maior R², mas modelo menos interpretável
- **Consideração**: Para análise política, interpretação é crucial

In [ ]:
# Vamos verificar algumas dessas preocupações no nosso modelo atual

print("=== DIAGNÓSTICOS DO MODELO MÚLTIPLO ===\n")

# 1. Verificar correlação entre variáveis independentes (multicolinearidade)
correlation_matrix = df_multiple[['perc_bolsa_familia', 'pib per capita']].corr()
print("1. CORRELAÇÃO ENTRE VARIÁVEIS INDEPENDENTES:")
print(f"Correlação Bolsa Família vs PIB per capita: {correlation_matrix.iloc[0,1]:.4f}")
if abs(correlation_matrix.iloc[0,1]) > 0.7:
    print("   ⚠️ ALERTA: Alta correlação! Risco de multicolinearidade")
elif abs(correlation_matrix.iloc[0,1]) > 0.5:
    print("   ⚠️ ATENÇÃO: Correlação moderada, monitorar")
else:
    print("   ✅ OK: Correlação baixa entre variáveis")

print(f"\n2. NÚMERO DE OBSERVAÇÕES vs VARIÁVEIS:")
n_obs = len(df_multiple)
n_vars = 2  # Bolsa Família + PIB per capita
ratio = n_obs / n_vars
print(f"Observações: {n_obs}")
print(f"Variáveis: {n_vars}")
print(f"Ratio obs/var: {ratio:.1f}")
if ratio < 10:
    print("   ⚠️ ALERTA: Poucas observações por variável")
elif ratio < 20:
    print("   ⚠️ ATENÇÃO: Ratio baixo, cuidado com overfitting")
else:
    print("   ✅ OK: Ratio adequado")

print(f"\n3. MELHORIA INCREMENTAL:")
improvement = test_r2_mult - test_r2
print(f"Melhoria no R²: {improvement:.4f}")
if improvement < 0.01:
    print("   ⚠️ ATENÇÃO: Melhoria muito pequena - variável pode não ser útil")
elif improvement < 0.05:
    print("   ✅ OK: Melhoria modesta mas válida")
else:
    print("   ✅ EXCELENTE: Melhoria significativa")

print(f"\n4. INTERPRETAÇÃO DOS COEFICIENTES:")
print(f"Bolsa Família: {model_multiple.coef_[0]:.4f} (cada 1% → +{model_multiple.coef_[0]:.2f}pp voto Lula)")
print(f"PIB per capita: {model_multiple.coef_[1]:.6f} (cada R$1.000 → {model_multiple.coef_[1]*1000:.3f}pp voto Lula)")

# 5. Verificar se os sinais fazem sentido teoricamente
print(f"\n5. CONSISTÊNCIA TEÓRICA:")
print(f"Bolsa Família positivo: {'✅ Esperado' if model_multiple.coef_[0] > 0 else '⚠️ Inesperado'}")
print(f"PIB per capita negativo: {'✅ Esperado' if model_multiple.coef_[1] < 0 else '⚠️ Inesperado'}")

### 📋 Recomendações para Próximas Variáveis

### **Baseado nos diagnósticos acima:**

✅ **Aspectos Positivos:**
- Sem problemas de multicolinearidade
- Coeficientes teoricamente consistentes
- Amostra grande o suficiente

⚠️ **Pontos de Atenção:**
- PIB per capita adiciona pouco poder explicativo
- Melhoria de apenas 0.4% no R²

### **Próximas variáveis a considerar (em ordem de prioridade):**

1. **Variáveis Dummy de Região** 
   - **Por quê**: Paradoxo de Simpson mostra diferenças regionais importantes
   - **Expectativa**: Melhoria significativa no R²

2. **Densidade Populacional** (ao invés de população bruta)
   - **Por quê**: Pode capturar efeitos urbano vs rural melhor

3. **Interação Bolsa Família × Região**
   - **Por quê**: Efeito do programa pode variar por região

### **Variáveis a EVITAR (por enquanto):**
- **Escolaridade**: Provavelmente correlacionada com PIB per capita
- **Taxa de desemprego**: Pode ser correlacionada com Bolsa Família
- **Idade média**: Sem teoria clara de relação com voto

### **Estratégia Recomendada:**
1. Adicionar uma variável por vez
2. Verificar melhoria no R²
3. Testar significância estatística
4. Manter apenas variáveis que agregam valor real

## Testando população

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error

# Multiple Linear Regression: % Bolsa Família + População Estimada


# Prepare data: drop NaNs for relevant columns
df_pop = merged_df[['perc_bolsa_familia', 'POPULAÇÃO ESTIMADA', 'voto_lula']].dropna()

X_pop = df_pop[['perc_bolsa_familia', 'POPULAÇÃO ESTIMADA']].values
y_pop = df_pop['voto_lula'].values

# Split into train and test sets
X_train_pop, X_test_pop, y_train_pop, y_test_pop = train_test_split(
    X_pop, y_pop, test_size=0.2, random_state=42
)

# Train model
model_pop = LinearRegression()
model_pop.fit(X_train_pop, y_train_pop)

# Predict
y_train_pred_pop = model_pop.predict(X_train_pop)
y_test_pred_pop = model_pop.predict(X_test_pop)

# Metrics
train_r2_pop = r2_score(y_train_pop, y_train_pred_pop)
test_r2_pop = r2_score(y_test_pop, y_test_pred_pop)
train_mae_pop = mean_absolute_error(y_train_pop, y_train_pred_pop)
test_mae_pop = mean_absolute_error(y_test_pop, y_test_pred_pop)

print("=== MODELO COM BOLSA FAMÍLIA + POPULAÇÃO ===\n")
print(f"Intercept: {model_pop.intercept_:.4f}")
print(f"Coeficiente Bolsa Família: {model_pop.coef_[0]:.4f}")
print(f"Coeficiente População Estimada: {model_pop.coef_[1]:.8f}")
print(f"\nR² (treino): {train_r2_pop:.4f}")
print(f"R² (teste): {test_r2_pop:.4f}")
print(f"MAE (treino): {train_mae_pop:.4f}")
print(f"MAE (teste): {test_mae_pop:.4f}")
print(f"\nDiferença R² (treino - teste): {train_r2_pop - test_r2_pop:.4f}")
print(f"Diferença MAE (teste - treino): {test_mae_pop - train_mae_pop:.4f}")

### Análise: Por que População não adiciona valor?

Exatamente como você observou - **população não adiciona praticamente nenhum valor ao modelo**. Vamos comparar com o modelo simples para quantificar isso:

In [ ]:
# Vamos comparar com o modelo simples (apenas Bolsa Família)
# First get the simple model results for comparison

# Simple model data (just Bolsa Família)
df_simple = merged_df[['perc_bolsa_familia', 'voto_lula']].dropna()
X_simple = df_simple[['perc_bolsa_familia']].values  
y_simple = df_simple['voto_lula'].values

# Split with same random_state for fair comparison
X_train_simple, X_test_simple, y_train_simple, y_test_simple = train_test_split(
    X_simple, y_simple, test_size=0.2, random_state=42
)

# Train simple model
model_simple = LinearRegression()
model_simple.fit(X_train_simple, y_train_simple)

# Predict
y_test_pred_simple = model_simple.predict(X_test_simple)
test_r2_simple = r2_score(y_test_simple, y_test_pred_simple)

print("=== COMPARAÇÃO DOS MODELOS ===\n")
print(f"Modelo simples (apenas Bolsa Família) - R² teste: {test_r2_simple:.4f}")
print(f"Modelo + população - R² teste: {test_r2_pop:.4f}")
print(f"Melhoria no R² teste: {test_r2_pop - test_r2_simple:.4f} ({((test_r2_pop - test_r2_simple)/test_r2_simple)*100:.1f}% de aumento)")

print(f"\n=== AVALIAÇÃO ===")
improvement = (test_r2_pop - test_r2_simple) * 100
if improvement < 0.5:
    print(f"❌ INSIGNIFICANTE: {improvement:.1f}pp de melhoria")
elif improvement < 2.0:
    print(f"⚠️  PEQUENA: {improvement:.1f}pp de melhoria")  
else:
    print(f"✅ SIGNIFICANTE: {improvement:.1f}pp de melhoria")

print(f"\n=== POR QUE POPULAÇÃO NÃO FUNCIONA? ===")
print(f"Coeficiente da população: {model_pop.coef_[1]:.8f}")
print("→ Coeficiente praticamente ZERO!")
print("→ População tem correlação muito fraca com voto no Lula")
print("→ Já está 'capturada' indiretamente pelo Bolsa Família")

In [ ]:
# Análise de correlação para entender melhor
import numpy as np

print("=== ANÁLISE DE CORRELAÇÃO ===")
corr_bf_lula = np.corrcoef(df_pop['perc_bolsa_familia'], df_pop['voto_lula'])[0,1]
corr_pop_lula = np.corrcoef(df_pop['POPULAÇÃO ESTIMADA'], df_pop['voto_lula'])[0,1]
corr_bf_pop = np.corrcoef(df_pop['perc_bolsa_familia'], df_pop['POPULAÇÃO ESTIMADA'])[0,1]

print(f"Correlação Bolsa Família ↔ Voto Lula: {corr_bf_lula:.3f}")
print(f"Correlação População ↔ Voto Lula: {corr_pop_lula:.3f}")
print(f"Correlação Bolsa Família ↔ População: {corr_bf_pop:.3f}")

print(f"\n=== CONCLUSÃO ===")
print(f"✅ Bolsa Família tem correlação FORTE com voto no Lula ({corr_bf_lula:.3f})")
print(f"❌ População tem correlação FRACA com voto no Lula ({corr_pop_lula:.3f})")
print(f"📊 O R² melhora apenas {(test_r2_pop - test_r2_simple)*100:.2f} pontos percentuais")
print(f"\n🎯 RECOMENDAÇÃO: Manter apenas o modelo simples com Bolsa Família")
print(f"   → Mais interpretável")
print(f"   → Performance praticamente idêntica") 
print(f"   → Evita overfitting com variáveis irrelevantes")

## Salvando Métricas e Modelo Inicial

Vamos salvar as principais métricas do modelo inicial para usar nas comparações dos próximos notebooks.

In [ ]:
# Salvando as principais métricas do Modelo Inicial
import pickle
import os

# Criar diretório para métricas se não existir
metrics_dir = os.path.join("..", "data", "metrics")
os.makedirs(metrics_dir, exist_ok=True)

# Métricas do modelo simples (apenas Bolsa Família)
modelo_inicial_metrics = {
    # Informações do modelo
    'modelo_nome': 'Modelo Inicial - Bolsa Família',
    'variaveis': ['perc_bolsa_familia'],
    'n_observacoes': len(df_lr),
    'n_variaveis': 1,
    
    # Coeficientes
    'intercept': model.intercept_,
    'coef_bolsa_familia': model.coef_[0],
    
    # Métricas de performance
    'r2_treino': train_r2,
    'r2_teste': test_r2,
    'mae_treino': train_mae,
    'mae_teste': test_mae,
    'mse_teste': mean_squared_error(y_test, y_test_pred),
    'rmse_teste': np.sqrt(mean_squared_error(y_test, y_test_pred)),
    
    # Diagnósticos
    'overfitting_r2_diff': train_r2 - test_r2,
    'overfitting_mae_diff': test_mae - train_mae,
    
    # Correlações
    'correlacao_bf_lula': merged_df[['perc_bolsa_familia', 'voto_lula']].corr().iloc[0,1],
    
    # Informações dos dados
    'voto_lula_min': merged_df['voto_lula'].min(),
    'voto_lula_max': merged_df['voto_lula'].max(),
    'voto_lula_mean': merged_df['voto_lula'].mean(),
    'bf_min': merged_df['perc_bolsa_familia'].min(),
    'bf_max': merged_df['perc_bolsa_familia'].max(),
    'bf_mean': merged_df['perc_bolsa_familia'].mean(),
    
    # Equação do modelo
    'equacao': f"% Voto Lula = {model.intercept_:.2f} + ({model.coef_[0]:.2f} × % Bolsa Família)"
}

# Salvar as métricas
metrics_file = os.path.join(metrics_dir, "modelo_inicial_metrics.pkl")
with open(metrics_file, 'wb') as f:
    pickle.dump(modelo_inicial_metrics, f)

print("=== MÉTRICAS DO MODELO INICIAL SALVAS ===\n")
print(f"📁 Arquivo: {metrics_file}")
print(f"📊 Modelo: {modelo_inicial_metrics['modelo_nome']}")
print(f"🎯 R² teste: {modelo_inicial_metrics['r2_teste']:.4f}")
print(f"📐 MAE teste: {modelo_inicial_metrics['mae_teste']:.4f}")
print(f"📈 Equação: {modelo_inicial_metrics['equacao']}")
print(f"🔗 Correlação BF-Lula: {modelo_inicial_metrics['correlacao_bf_lula']:.4f}")

print(f"\n✅ Métricas salvas com sucesso!")
print(f"   → Use nos próximos notebooks para comparações")
print(f"   → Arquivo pickle pode ser carregado com pd.read_pickle()")

# Verificar se o arquivo foi criado
if os.path.exists(metrics_file):
    print(f"\n📋 Arquivo confirmado: {os.path.getsize(metrics_file)} bytes")
else:
    print(f"\n❌ ERRO: Arquivo não foi criado!")

In [ ]:
# Função auxiliar para carregar métricas nos outros notebooks
def load_modelo_inicial_metrics():
    """
    Função para carregar as métricas do modelo inicial nos outros notebooks.
    
    Returns:
        dict: Dicionário com todas as métricas do modelo inicial
    """
    import pickle
    import os
    
    metrics_file = os.path.join("..", "data", "metrics", "modelo_inicial_metrics.pkl")
    
    if not os.path.exists(metrics_file):
        raise FileNotFoundError(f"Arquivo de métricas não encontrado: {metrics_file}")
    
    with open(metrics_file, 'rb') as f:
        metrics = pickle.load(f)
    
    return metrics

# Salvar a função em um arquivo Python para importação
import os
utils_dir = os.path.join("..", "utils")
os.makedirs(utils_dir, exist_ok=True)

# Criar arquivo utils/model_utils.py
utils_content = '''"""
Utilitários para carregar métricas dos modelos de regressão
"""
import pickle
import os

def load_modelo_inicial_metrics():
    """
    Função para carregar as métricas do modelo inicial nos outros notebooks.
    
    Returns:
        dict: Dicionário com todas as métricas do modelo inicial
    """
    metrics_file = os.path.join("..", "data", "metrics", "modelo_inicial_metrics.pkl")
    
    if not os.path.exists(metrics_file):
        raise FileNotFoundError(f"Arquivo de métricas não encontrado: {metrics_file}")
    
    with open(metrics_file, 'rb') as f:
        metrics = pickle.load(f)
    
    return metrics
'''

utils_file = os.path.join(utils_dir, "model_utils.py")
with open(utils_file, 'w', encoding='utf-8') as f:
    f.write(utils_content)

print("=== FUNÇÃO EXPORTADA PARA IMPORTAÇÃO ===")
print(f"📁 Arquivo criado: {utils_file}")
print(f"📦 Agora você pode importar nos outros notebooks:")
print(f"   from utils.model_utils import load_modelo_inicial_metrics")

# Exemplo de como usar nos outros notebooks
print("\n=== COMO USAR NOS OUTROS NOTEBOOKS ===")
print("""
# No início dos outros notebooks, adicione:
import sys
import os
sys.path.append('..')
from utils.model_utils import load_modelo_inicial_metrics

# Usar as métricas
inicial_metrics = load_modelo_inicial_metrics()
print(f"Modelo inicial R²: {inicial_metrics['r2_teste']:.4f}")
print(f"Modelo inicial equação: {inicial_metrics['equacao']}")
""")

# Testar a função
print("\n=== TESTE DA FUNÇÃO ===")
try:
    test_metrics = load_modelo_inicial_metrics()
    print(f"✅ Função funciona! R² = {test_metrics['r2_teste']:.4f}")
except Exception as e:
    print(f"❌ Erro ao testar função: {e}")

print(f"\n🎯 PRONTO PARA OS PRÓXIMOS NOTEBOOKS!")
print(f"   → As métricas estão salvas em: modelo_inicial_metrics.pkl")
print(f"   → Use: from utils.model_utils import load_modelo_inicial_metrics")
print(f"   → Muito mais limpo que redefinir a função toda vez!")

In [ ]:
import pickle
import os

# Salvar o modelo treinado (apenas Bolsa Família) como pickle

model_dir = os.path.join("..", "data", "models")
os.makedirs(model_dir, exist_ok=True)

model_file = os.path.join(model_dir, "modelo_inicial_bolsa_familia.pkl")
with open(model_file, 'wb') as f:
    pickle.dump(model, f)

print(f"✅ Modelo salvo em: {model_file}")